<a href="https://colab.research.google.com/github/guilhermelaviola/MovieRecommendation/blob/main/MovieRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing all the necessary libraries:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel

In [2]:
# Importing the 'credits' and 'movies' datasets:
credits = pd.read_csv('tmdb_5000_credits.csv')
movies = pd.read_csv('tmdb_5000_movies.csv')

In [3]:
# Diplaying both datasets:
print(credits.head(10))
print(movies.head(10))

   movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   
5       559                              Spider-Man 3   
6     38757                                   Tangled   
7     99861                   Avengers: Age of Ultron   
8       767    Harry Potter and the Half-Blood Prince   
9    209112        Batman v Superman: Dawn of Justice   

                                                cast  \
0  [{"cast_id": 242, "character": "Jake Sully", "...   
1  [{"cast_id": 4, "character": "Captain Jack Spa...   
2  [{"cast_id": 1, "character": "James Bond", "cr...   
3  [{"cast_id": 2, "character": "Bruce Wayne / Ba...   
4  [{"cast_id": 5, "character": "John Carter", "c...   
5  [{"cast_id": 30, "character": "Pe

In [4]:
# Displaying both datasets shapes (table shapes):
print('Credits dataframe: ', credits.shape)
print('Movies dataframe: ', movies.shape)

Credits dataframe:  (4803, 4)
Movies dataframe:  (4803, 20)


In [12]:
credits_column_renamed = credits.rename(index = str,
                                        columns = {'movie_id' : 'id'})
movies_merge = movies.merge(credits_column_renamed, on = 'id')
print(movies_merge.head())

      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id": 470, "nam

In [13]:
movies_cleaned = movies_merge.drop(columns = ['homepage',
                                              'title_x',
                                              'title_y',
                                              'status',
                                              'production_countries'])
print(movies_cleaned.head())
print(movies_cleaned.info())
print(movies_cleaned.head(1)['overview'])

      budget                                             genres      id  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   19995   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...     285   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...  206647   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   49026   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id": 470, "name": "spy"}, {"id": 818, "name...                en   
3  [{"id": 849, "name": "dc comics"}, {"id": 853,...                en   
4  [{"id": 818, "name": "based on novel"}, {"id":...                en   

                             original_title  \
0                                    Avatar   
1  Pirates

In [16]:
tfv = TfidfVectorizer(min_df = 3,
                      max_features = None,
                      strip_accents = 'unicode',
                      analyzer = 'word',
                      token_pattern = r'\w{1,}',
                      ngram_range = (1, 3),
                      stop_words = 'english')

In [17]:
# Fitting the TF_IDF on the 'overview' text:
tfv_matrix = tfv.fit_transform(movies_cleaned['overview'])
print(tfv_matrix)
print(tfv_matrix.shape)

ValueError: ignored

In [20]:
# Computing the Sigmoid Kernel
sigmoid = sigmoid_kernel(tfv_matrix, tfv_matrix)
print(sigmoid[0])

NameError: ignored

In [21]:
# Reverse mapping of indexes and movie titles:
indexes = pd.Series(movies_cleaned.index,
                    index = movies_cleaned['original_title']).drop_duplicates()
print(indexes)
print(indexes['Newlyweds'])
print(sig[4799])
print(list(enumerate(sig[indexes['Newlyweds']])))
print(sorted(list(enumerate(sig[indexes['Newlyweds']])),
             key = lambda x : x[1], reverse = True))

original_title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64
4799


NameError: ignored

In [ ]:
def give_recommendations(title, sigmoid = sigmoid):
  # Getting the index corresponding to original_title:
  index = indexes[title]

  # Getting the pairwise similarity scores:
  sigmoid_scores = list(enumerate(sigmoid[index]))

  # Sorting the movies:
  sigmoid_scores = sorted(sigmoid_scores, key = lambda x : x[1], reverse = True)

  # Scores of the 10 most similar movies:
  sigmoid_scores = sigmoid_scores[1:11]

  # Movie indexes:
  movie_indexes = [i[0] for i in sigmoid_scores]

  # Top 10 most similar movies:
  return movies_cleaned['original_title'].iloc[movie_indexes]